# Run Model Inference

In [1]:
# setup config
import os
from PIL import Image
import torch
from maskrcnn_benchmark.data import make_data_loader
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
from maskrcnn_benchmark.data import transforms as T
from maskrcnn_benchmark.config import try_to_find
from maskrcnn_benchmark.structures.bounding_box import BoxList
from maskrcnn_benchmark.utils.miscellaneous import mkdir, save_config
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.data.transforms import build_transforms # input image and box transforms

/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/deform_conv.py:313: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/deform_conv.py:378: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/deform_conv.py:428: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/layers/set_loss.py:144: FutureWarning: `torch.cuda.amp.custom_f

In [44]:
from maskrcnn_benchmark.engine.inference import create_queries_and_maps_from_dataset, resize_box

In [6]:
# set other args -- 
def get_cfg():
    """MQ-GLIP-L config"""
    from maskrcnn_benchmark.config import cfg
    cfg.local_rank = 0
    cfg.num_gpus = 1
    config_file = "configs/pretrain/mq-glip-l.yaml"
    cfg.merge_from_file(config_file)
    additional_model_config = "configs/vision_query_5shot/lvis_minival_L.yaml"
    cfg.merge_from_file(additional_model_config)
    
    opts = [
        "MODEL.WEIGHT", "MODEL/mq-glip-l",
        "TEST.IMS_PER_BATCH", "1",
        "VISION_QUERY.QUERY_BANK_PATH", "MODEL/lvis_query_5_pool7_sel_large.pth",
    ]
    cfg.merge_from_list(opts)
    
    cfg.freeze()
    return cfg

def build_model(cfg):
    model = build_detection_model(cfg)
    model.to(cfg.MODEL.DEVICE)
    checkpointer = DetectronCheckpointer(cfg, model, save_dir=cfg.OUTPUT_DIR)
    _ = checkpointer.load(cfg.MODEL.WEIGHT, force=True)
    model.eval()
    return model

In [7]:
cfg = get_cfg()
model = build_model(cfg)

VISION BACKBONE USE GRADIENT CHECKPOINTING:  False


/home/jielei/miniconda/envs/ov_det/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


LANGUAGE BACKBONE USE GRADIENT CHECKPOINTING:  False


Some weights of QVBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['encoder.qv_layer.0.attn.norm.bias', 'encoder.qv_layer.0.attn.norm.weight', 'encoder.qv_layer.0.attn.norm_kv.bias', 'encoder.qv_layer.0.attn.norm_kv.weight', 'encoder.qv_layer.0.attn.to_kv.weight', 'encoder.qv_layer.0.attn.to_out.weight', 'encoder.qv_layer.0.attn.to_q.weight', 'encoder.qv_layer.0.attn_gate.linear1.weight', 'encoder.qv_layer.0.attn_gate.linear2.weight', 'encoder.qv_layer.0.attn_gate.norm.bias', 'encoder.qv_layer.0.attn_gate.norm.weight', 'encoder.qv_layer.0.ff.linear1.weight', 'encoder.qv_layer.0.ff.linear2.weight', 'encoder.qv_layer.0.ff.norm.bias', 'encoder.qv_layer.0.ff.norm.weight', 'encoder.qv_layer.0.ff_gate', 'encoder.qv_layer.1.attn.norm.bias', 'encoder.qv_layer.1.attn.norm.weight', 'encoder.qv_layer.1.attn.norm_kv.bias', 'encoder.qv_layer.1.attn.norm_kv.weight', 'encoder.qv_layer.1.attn.to_kv.weight', 'encoder.qv_layer.1.attn.to_out.weight

EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B


Some layers unloaded with pre-trained weight: 
rpn.head.cls_logits.{bias, weight}


In [11]:
iou_types = ("bbox",)
data_loader = make_data_loader(cfg, is_train=False, is_distributed=False)[0]
dataset = data_loader.dataset

The combined datasets are: ('lvis_evaluation_mini_val',).
Loading annotations.
Done (t=0.77s)
Creating index.
Index created.
lvis_evaluation_mini_val has the 4809 data points LvisDetection


In [13]:
device = torch.device(cfg.MODEL.DEVICE)
cpu_device = torch.device("cpu")

In [18]:
results_dict = {}

In [19]:
# Get a single batch
batch_iterator = iter(data_loader)
batch_data = next(batch_iterator)
print("Batch data:", batch_data)

Batch data: (<maskrcnn_benchmark.structures.image_list.ImageList object at 0x73981c104d90>, ({'boxes': tensor([[606.9400, 303.3400, 639.0000, 356.8600],
        [166.8900, 233.5800, 185.3500, 265.4400],
        [398.5100, 205.1000, 405.6900, 215.2200],
        [550.2200, 309.1400, 583.8000, 400.2400],
        [240.8800, 197.0000, 252.0500, 211.8700],
        [338.8600, 199.6600, 347.3100, 214.0500],
        [351.1400, 207.7800, 360.5100, 229.7400],
        [  0.0000,  38.9300, 172.1600, 128.8500],
        [512.5800, 225.7200, 539.6500, 274.2700],
        [480.1000, 223.5200, 491.0800, 277.0700],
        [466.6700, 222.9000, 479.0300, 278.6700],
        [479.4500, 171.7000, 490.7400, 222.1200],
        [511.4800, 141.7700, 524.3900, 173.9000],
        [  0.0000, 276.4800, 133.9300, 333.2600],
        [489.1100, 172.5700, 511.5500, 283.0500],
        [562.4900, 212.2500, 639.0000, 283.0200],
        [555.9100, 206.4100, 639.0000, 288.3400],
        [  5.4100, 164.9500, 154.0800, 262.2200

In [26]:
images, targets, image_ids, *_ = batch_data
print(f"images.tensors.shape {images.tensors.shape}")
print(f"len(targets) {len(targets)}")
print(f"targets[0]['boxes'].shape {targets[0]['boxes'].shape}")
print(f"targets[0]['labels'].shape {targets[0]['labels'].shape}")
print(f"image_ids {image_ids}")

images.tensors.shape torch.Size([1, 3, 800, 1216])
len(targets) 1
targets[0]['boxes'].shape torch.Size([18, 4])
targets[0]['labels'].shape torch.Size([18])
image_ids (0,)


In [30]:
# todo
all_queries, all_positive_map_label_to_token = \
    create_queries_and_maps_from_dataset(
        dataset, cfg, disable_print=True
    )

In [32]:
# all_queries are the concatenation of all query text
print(f"len(all_queries), {len(all_queries)}")
print(f"all_queries[0], {all_queries[0]}")

len(all_queries), 31
all_queries[0], aerosol can. air conditioner. airplane. alarm clock. alcohol. alligator. almond. ambulance. amplifier. anklet. antenna. apple. applesauce. apricot. apron. aquarium. arctic . armband. armchair. armoire. armor. artichoke. trash can. ashtray. asparagus. atomizer. avocado. award. awning. ax. baboon. baby buggy. basketball backboard. backpack. handbag. suitcase. bagel. bagpipe. baguet. bait


In [35]:
print(f"len(all_positive_map_label_to_token) {len(all_positive_map_label_to_token)}")
print(f"len(all_positive_map_label_to_token[0]) {len(all_positive_map_label_to_token[0])}")

len(all_positive_map_label_to_token) 31
len(all_positive_map_label_to_token[0]) 40


In [37]:
query_i = 0
captions = [all_queries[query_i] for ii in range(len(targets))]
positive_map_label_to_token = all_positive_map_label_to_token[query_i]

### Run inference with internal forward function

In [40]:
output = model(
    images.to(device), 
    captions=captions, 
    positive_map=positive_map_label_to_token
)

/home/jielei/miniconda/envs/ov_det/lib/python3.10/site-packages/transformers/modeling_utils.py:1614: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/jielei/MQ-Det/maskrcnn_benchmark/modeling/rpn/vldyhead.py:224: UserWarning: `nn.functional.upsample_bilinear` is deprecated. Use `nn.functional.interpolate` instead.
  temp_fea.append(F.upsample_bilinear(self.DyConv[0](visual_feats[level + 1], **conv_args),


In [41]:
output = [o.to(cpu_device) for o in output]
# len(output) == 1

In [47]:
mdetr_style_output = []

In [45]:
# output = output[0]
output = resize_box(output, targets)
scores = output.extra_fields["scores"]
labels = output.extra_fields["labels"]
boxes = output.bbox
mdetr_style_output.append((targets[0]["image_id"].item(), {"scores": scores, "labels": labels, "boxes": boxes}))

NameError: name 'mdetr_style_output' is not defined

In [51]:
len(scores)

90

In [50]:
scores

tensor([0.2666, 0.2419, 0.2098, 0.1999, 0.2339, 0.2872, 0.5625, 0.2698, 0.2341,
        0.1572, 0.1952, 0.1874, 0.1961, 0.3257, 0.2210, 0.2717, 0.2681, 0.1921,
        0.1942, 0.2086, 0.2601, 0.2066, 0.2483, 0.3237, 0.3611, 0.4103, 0.5202,
        0.2265, 0.2481, 0.1909, 0.1836, 0.2003, 0.2370, 0.2585, 0.2503, 0.3306,
        0.5526, 0.3465, 0.2616, 0.3472, 0.2737, 0.2313, 0.1800, 0.3191, 0.1800,
        0.2381, 0.2223, 0.2420, 0.2628, 0.1864, 0.1883, 0.1398, 0.2173, 0.2172,
        0.3725, 0.1822, 0.2366, 0.2352, 0.1973, 0.3507, 0.3331, 0.2312, 0.4629,
        0.1637, 0.1980, 0.2136, 0.2176, 0.1989, 0.2993, 0.2361, 0.3761, 0.3259,
        0.2289, 0.3363, 0.2754, 0.2352, 0.4140, 0.2154, 0.2961, 0.2177, 0.2608,
        0.2092, 0.1456, 0.1459, 0.2619, 0.1689, 0.1640, 0.2152, 0.4973, 0.2288],
       grad_fn=<ToCopyBackward0>)

In [52]:
len(labels)

90

In [53]:
labels

tensor([ 2, 11, 12, 20,  2,  9, 20, 20, 20,  1, 15, 20,  5, 15, 12, 20,  9, 20,
        36,  5,  5, 15,  1,  5, 20, 20, 20, 15, 20, 20, 10, 35,  9, 19, 23, 34,
        35, 36, 19, 23, 35, 36,  9, 19, 35, 23, 35,  9, 36, 35, 35, 35,  2, 20,
        20, 15, 20, 20, 19, 20, 15, 20, 20, 20, 15, 20, 20, 20, 20, 19, 20, 20,
        20,  5,  2,  9, 20, 20,  9, 19, 20, 20, 15, 19, 20, 20, 20, 20, 20, 20])

In [54]:
len(boxes)

90

In [55]:
boxes

tensor([[5.0545e+02, 8.3904e+01, 5.3385e+02, 1.3484e+02],
        [5.0545e+02, 8.3904e+01, 5.3385e+02, 1.3484e+02],
        [4.4787e+02, 1.2042e+02, 4.6073e+02, 1.4120e+02],
        [4.7615e+02, 1.3679e+02, 5.0348e+02, 1.7144e+02],
        [4.7603e+02, 1.3669e+02, 5.2467e+02, 1.7442e+02],
        [4.7603e+02, 1.3669e+02, 5.2467e+02, 1.7442e+02],
        [4.7603e+02, 1.3669e+02, 5.2467e+02, 1.7442e+02],
        [4.9537e+02, 1.3778e+02, 5.2486e+02, 1.7410e+02],
        [5.0973e+02, 1.4198e+02, 5.2443e+02, 1.7383e+02],
        [4.9246e+02, 1.5416e+02, 5.0109e+02, 1.7234e+02],
        [4.2731e+02, 1.7353e+02, 4.5004e+02, 2.2629e+02],
        [4.7877e+02, 1.7135e+02, 4.9130e+02, 2.2235e+02],
        [3.1363e+02, 1.9161e+02, 3.2378e+02, 2.1420e+02],
        [4.2980e+02, 1.7344e+02, 4.6224e+02, 2.3188e+02],
        [3.9813e+02, 2.0486e+02, 4.0642e+02, 2.1618e+02],
        [4.9144e+02, 1.7289e+02, 5.1195e+02, 2.4199e+02],
        [5.1302e+02, 2.0552e+02, 5.2655e+02, 2.2100e+02],
        [5.130

### Run inference without internal forward

In [59]:
# def run_inference_wrapper(images, captions, visuql_queries, model):

images = images.to(device)
device = images.tensors.device
visual_features = model.backbone(images.tensors)
assert images.tensors.shape[0]==1 # TODO: Only query batch size = 1 for test
labels_in_caption, all_map = model.get_labels_and_maps_from_positive_map(
    positive_map=positive_map_label_to_token, dtype=visual_features[0].dtype)
batched_labels_in_caption = [labels_in_caption]
batched_all_map = [all_map]
batched_pos_category_map = None
batched_pos_labels = None    
# query_features, query_attetion_masks, batched_has_vision_query=self.query_selector(
#     batched_labels_in_caption, batched_all_map, batched_pos_labels)

In [63]:
batched_label_list, batched_location_map, batched_pos_labels = \
    batched_labels_in_caption, batched_all_map, batched_pos_labels

In [68]:
@torch.no_grad()
def extract_visual_prompt_feature(t_image_list, box_list, model):
    """
    image_list: ImageList
    box_list: List[BoxList]
    """
    images = to_image_list(image_list)
    visual_features = model.backbone(image_list.tensors)    
    query_feats=model.pooler(visual_features, box_list) # num_boxes, num_channels, pooler_size, pooler_size
    query_feats=query_feats[None, ...] # 1, num_boxes, num_channels, pooler_size, pooler_size
    query_feats = query_feats.mean(dim=[-2,-1]).permute(1, 0, 2) # num_boxes, num_scales, num_channels
    # labels=torch.cat([t.get_field('labels') for t in box_list])
    # max_query_number = cfg.VISION_QUERY.MAX_QUERY_NUMBER
    # return query_feats, labels
    return query_feats

[[1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40]]

In [61]:
all_map

tensor([[0.0000, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')

In [62]:
all_map.shape

torch.Size([40, 256])

In [66]:
all_map[0]

tensor([0.0000, 0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [67]:
all_map[1]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333, 0.3333, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [60]:
batched_labels_in_caption[0]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40]